In [43]:
# Statistics
import pandas as pd
import numpy as np
import math as mt

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Data Preprocessing - Standardization, Encoding, Imputation
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer


# Data Preprocessing - Feature Engineering
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA

# Data Preprocessing - ML Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# ML - Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# ML - Evaluation
from sklearn.model_selection import cross_val_score

# ML - Tuning
from sklearn.model_selection import GridSearchCV

In [53]:
# Read train dataset
train_data = pd.read_csv('../input/30days-folds/train_folds.csv')
test_data = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

In [18]:
%%time
# No Standardization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    #print("encoding")
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    #print("training")
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7245705537554137
1 0.7242510333821858
2 0.7270667092065692
3 0.7268359229595335
4 0.7257178555909586
0.7256884149789322 0.0011430674400777338
CPU times: user 22.6 s, sys: 771 ms, total: 23.3 s
Wall time: 22.5 s


In [19]:
%%time
# With Standardization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Preprocessing - Standardization
    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7241755479182882
1 0.7241138968948254
2 0.7267386816038165
3 0.7268357864120136
4 0.725667388462628
0.7255062602583143 0.001185068397378747
CPU times: user 22.9 s, sys: 666 ms, total: 23.5 s
Wall time: 23 s


In [25]:
%%time
# With Normalization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform

    # Preprocessing - Normalizatino
    normalizer = Normalizer()
    X_train[num_cols] = normalizer.fit_transform(X_train[num_cols])
    X_valid[num_cols] = normalizer.transform(X_valid[num_cols])
    X_test[num_cols] = normalizer.transform(X_test[num_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7388116889933081
1 0.738024917853026
2 0.7409186250111057
3 0.7396434860172969
4 0.740614097083746
0.7396025629916967 0.0010836365642808196
CPU times: user 23 s, sys: 829 ms, total: 23.8 s
Wall time: 23.1 s


In [26]:
%%time
# With Standardization + Normalization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform

    # Preprocessing - Standardization
    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols]) # Q. The last transform
    
    # Preprocessing - Normalizatino
    normalizer = Normalizer()
    X_train[num_cols] = normalizer.fit_transform(X_train[num_cols])
    X_valid[num_cols] = normalizer.transform(X_valid[num_cols])
    X_test[num_cols] = normalizer.transform(X_test[num_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7347795628127367
1 0.7354956594532642
2 0.7363710377512285
3 0.7369826624019941
4 0.7375458010938692
0.7362349447026185 0.0009960548860008741
CPU times: user 23.4 s, sys: 694 ms, total: 24.1 s
Wall time: 23.4 s


In [ ]:
%%time
# With Standardization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Preprocessing - Standardization
    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [27]:
%%time
# Log transformation + Tuning
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

for col in num_cols:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

print('You need to reset dataframe!')

0 0.7187408168536737
1 0.7184891153933534
2 0.7204486544876513
3 0.7204357985914301
4 0.7191577004325692
0.7194544171517355 0.0008343489787694719
CPU times: user 28.8 s, sys: 853 ms, total: 29.6 s
Wall time: 28.9 s


In [47]:
%%time
# polynomial features + Tuning
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

poly = PolynomialFeatures(degree=2, 
                          interaction_only=True, # If true, only interaction features are produced: features that are products of at most degree distinct input features (so not x[1] ** 2, x[0] * x[2] ** 3, etc.).
                          include_bias=False)
train_poly = poly.fit_transform(train_data[num_cols])
test_poly = poly.fit_transform(test_data[num_cols])

df_train_poly = pd.DataFrame(train_poly, columns=[f"poly_{i}" for i in range(train_poly.shape[1])])
df_test_poly = pd.DataFrame(test_poly, columns=[f"poly_{i}" for i in range(test_poly.shape[1])])

train_data = pd.concat([train_data, df_train_poly], axis=1)
test_data = pd.concat([test_data, df_test_poly], axis=1)

useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

print('You need to reset dataframe!')

0 0.7205875231523107
1 0.7204528173429748
2 0.7224474858735078
3 0.7217891599660472
4 0.7210479916863682
0.7212649956042417 0.0007534885090919923
You need to reset dataframe!
CPU times: user 52.2 s, sys: 4 s, total: 56.2 s
Wall time: 53.5 s


In [59]:
test_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,E,I,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,A,B,A,C,B,C,A,E,C,H,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,B,A,A,A,B,B,A,E,D,K,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,B,B,A,C,B,D,A,E,A,N,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,B,B,A,C,B,C,A,E,C,F,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,B,A,A,C,B,D,A,E,E,I,...,0.287454,0.543800,0.682378,1.028978,1.022741,0.683903,0.877273,0.532410,0.605397,0.884581
199996,B,A,A,C,B,B,A,E,C,F,...,0.794881,0.432778,0.389775,0.359871,0.550013,0.492082,0.202295,0.416875,0.406205,0.758665
199997,A,B,B,C,B,B,A,E,C,I,...,0.514487,0.060997,0.171741,0.317185,0.150340,0.122109,0.390524,0.334026,0.378987,0.839416
199998,A,A,A,C,B,D,A,D,A,F,...,0.286144,1.061710,0.819811,0.901241,0.555339,0.844315,0.894193,0.794102,0.844279,0.890473


In [89]:
%%time
# One-Hot Encoding + Ordinal Encoding + Tuning
# pd.cut 
# Model Tuning + drop cat2
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
oe_cols = ['cat9']
ohe_cols = cat_cols
ohe_cols.remove('cat9')
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[oe_cols] = ordinal_encoder.fit_transform(X_train[oe_cols])
    X_valid[oe_cols] = ordinal_encoder.transform(X_valid[oe_cols])
    X_test[oe_cols] = ordinal_encoder.transform(X_test[oe_cols]) # Q. The last transform
    
    # Preprocessing - One-Hot Encoding
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
    X_train_ohe = ohe.fit_transform(X_train[ohe_cols])
    X_valid_ohe = ohe.transform(X_valid[ohe_cols])
    X_test_ohe = ohe.transform(X_test[ohe_cols]) # Q. The last transform
    
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=[f"ohe_{i}" for i in range(X_train_ohe.shape[1])])
    X_valid_ohe = pd.DataFrame(X_valid_ohe, columns=[f"ohe_{i}" for i in range(X_valid_ohe.shape[1])])
    X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])
    
    X_train = pd.concat([X_train.drop(columns=ohe_cols), X_train_ohe], axis=1)
    X_valid = pd.concat([X_valid.drop(columns=ohe_cols), X_valid_ohe], axis=1)
    X_test = pd.concat([X_test.drop(columns=ohe_cols), X_test_ohe], axis=1)

    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

print('You need to reset dataframe!')

0 0.71869890821944
1 0.7184030212315331
2 0.7204136662080575
3 0.7203299560598941
4 0.7191436344980804
0.719397837243401 0.0008298835686080188
You need to reset dataframe!
CPU times: user 27.7 s, sys: 1.83 s, total: 29.5 s
Wall time: 28.1 s


In [32]:
%%time
# Model Tuning + drop cat2
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold", "cat2")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    #model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

print('You need to reset dataframe!')

0 0.718609466420021
1 0.7185071559268716
2 0.7202768090254914
3 0.7204231710582014
4 0.719266186984806
0.7194165578830783 0.0008067616831622805
CPU times: user 26.9 s, sys: 676 ms, total: 27.6 s
Wall time: 26.9 s


In [36]:
%%time
# Model Tuning + drop cat2, cat6
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold", "cat2", "cat6")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    #model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

print('You need to reset dataframe!')

0 0.7187555969676352
1 0.7184220257043639
2 0.7204441725501761
3 0.7202437427874456
4 0.719224667690094
0.7194180411399429 0.0008003750623549543
CPU times: user 24.6 s, sys: 675 ms, total: 25.3 s
Wall time: 24.6 s


In [61]:
%%time
# Tuning + Standardization
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Preprocessing - Standardization
    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    #model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7188216743231635
1 0.7185658026731828
2 0.7204336784139413
3 0.7202232260837255
4 0.719103007983702
0.7194294778955429 0.0007563593779478354
CPU times: user 28.3 s, sys: 410 ms, total: 28.7 s
Wall time: 28 s


In [31]:
%%time
# Only Model Tuning
useful_features = [col for col in train_data.columns if col not in ("id", "target", "kfold")]
cat_cols = [col for col in useful_features if "cat" in col]
num_cols = [col for col in useful_features if col.startswith('cont')]
test_data = test_data[useful_features]

final_predictions = []
scores = []

for fold in range(5):
    # Preprocessing - Kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    # Preprocessing - Ordinal Encoding
    ordinal_encoder = OrdinalEncoder()
    X_train[cat_cols] = ordinal_encoder.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = ordinal_encoder.transform(X_valid[cat_cols])
    X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols]) # Q. The last transform
    
    # Training
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1)
    #model = XGBRegressor(random_state=fold, n_jobs=8)
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor', 
                         learning_rate=0.1, n_estimators=1000, max_depth=3, colsample_bytree=0.3)
    model.fit(X_train, y_train)
    
    # Evaluation
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7187526657413477
1 0.7183753362008053
2 0.7204385881841233
3 0.7202580864662346
4 0.7190828534147123
0.7193815060014447 0.0008225359761356826
CPU times: user 28.5 s, sys: 792 ms, total: 29.3 s
Wall time: 28.5 s


In [29]:
# Export submission.csv
preds = np.mean(np.column_stack(final_predictions), axis=1)
preds = pd.DataFrame({'id': sample_submission.id, 'target': preds})
preds.to_csv('submission.csv', index=False)